In [2]:
import sys 
sys.path.append('../../')
from src.import_packages import *
from src.visualize_EDA import vEDA

from bs4 import BeautifulSoup
import requests

from geopy.geocoders import Nominatim, ArcGIS
geolocator = Nominatim(user_agent = 'user_agent', timeout = 10)

def geocode_location(location):

    try:
        geocoded = geolocator.geocode(location)
        return (geocoded.latitude, geocoded.longitude)
    
    except:
        geocoded = ArcGIS().geocode(location)
        return (geocoded.latitude, geocoded.longitude)    


def plot_map(data, _name = 'Lidl'):

    data['size'] = 10

    fig = px.scatter_mapbox(data, lat = 'latitude', lon = 'longitude', hover_data=['store'],
                                title = _name + ' boltok elhelyezkedése', 
                                center = {"lat": data['latitude'].mean(), "lon": data['longitude'].mean()},
                                zoom = 6, size_max = 7, size = 'size',
                                mapbox_style = 'carto-positron', height = 650, width = 950)
    fig.show()

    data.drop('size', 1, inplace = True)

Importing the following packages: pandas, numpy, matplotlib, seaborn, plotly, warnings


## Lidl

In [6]:
url = 'https://www.lidl.hu/extrak/uezleteink'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [19]:
lidl_stores = []

for i in soup.find_all('div', class_ = 'ezrichtext-field'):
    for j in i.find_all('li'):
        lidl_stores.append(j.text)

In [21]:
print('There are', len(lidl_stores), 'Lidl stores in HU')

There are 186 Lidl stores in HU


Geocode locations

In [22]:
lidl_stores[:3]

['Agárd, Akácfa utca 2.',
 'Ajka, Hársfa utca 1/A',
 'Baja, Bajcsy-Zsilinszky utca 9.']

In [55]:
lidl_coords = {}
not_found = []

for lidl_store in lidl_stores:

    try:
        lidl_coords[lidl_store] = geocode_location(lidl_store)

    except AttributeError:
        not_found.append(lidl_store)
        lidl_coords[lidl_store] = np.nan

In [57]:
not_found

['Sárbogárd, Ady E. utca 232-236.']

In [81]:
lidl_coords['Sárbogárd, Ady E. utca 232-236.'] = (46.87754672721678, 18.625487244828157)
lidl_coords['Barcs, Erkel F. utca 2.'] = (45.96236347916862, 17.467622603785895)

Check on map and correct if there are mistakes

In [82]:
lidl_coords_df = pd.DataFrame(lidl_coords).T.rename_axis('store').reset_index()
lidl_coords_df.columns = ['store', 'latitude', 'longitude']

In [88]:
plot_map(lidl_coords_df)

## Aldi